In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os
import math


2025-10-26 21:21:50.338733: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-26 21:21:50.580965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-26 21:21:52.102463: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10  # You might want to increase this for better accuracy
NUM_CLASSES = 7 # Angry, Disgusted, Happy, Normal, Sad, Scared, Surprised
DATASET_PATH = "Cat Emotions.v1-test.folder"
TRAIN_DIR = os.path.join(DATASET_PATH, "train")

MODEL_SAVE_PATH = "cat_emotion_mobilenet_v2_v1_test.h5"

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1 # 10% for validation
)

# Create the generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', # For one-hot encoded labels
    subset='training', # Set as training data
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', # For one-hot encoded labels
    subset='validation', # Set as validation data
    shuffle=False
)

# Load MobileNetV2 pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Add custom top layers for single-label classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x) # Added a new Dense layer
x = Dropout(0.5)(x) # Added a Dropout layer for regularization
x = Dense(NUM_CLASSES, activation='softmax')(x) # Softmax for single-label classification

# Create the new model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # Use categorical_crossentropy for single-label classification
              metrics=['accuracy'])

# Train the model
print("Starting model training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=math.ceil(train_generator.samples / BATCH_SIZE),
    validation_data=validation_generator,
    validation_steps=math.ceil(validation_generator.samples / BATCH_SIZE),
    verbose=1
)
print("Model training finished.")

# Save the trained model
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")


Found 609 images belonging to 7 classes.
Found 62 images belonging to 7 classes.


2025-10-26 21:21:59.223987: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Starting model training...


/home/qod110/anaconda3/lib/python3.13/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 603ms/step - accuracy: 0.1888 - loss: 2.1444 - val_accuracy: 0.2742 - val_loss: 1.7521
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 537ms/step - accuracy: 0.2989 - loss: 1.7994 - val_accuracy: 0.4355 - val_loss: 1.6139
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 535ms/step - accuracy: 0.3662 - loss: 1.6718 - val_accuracy: 0.4355 - val_loss: 1.5493
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 540ms/step - accuracy: 0.4154 - loss: 1.5858 - val_accuracy: 0.4677 - val_loss: 1.4967
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 528ms/step - accuracy: 0.4187 - loss: 1.5278 - val_accuracy: 0.4677 - val_loss: 1.4592
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 526ms/step - accuracy: 0.4236 - loss: 1.4886 - val_accuracy: 0.5323 - val_loss: 1.3767
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 531ms/step - accuracy: 0.4943 - loss: 1.3759 - val_accuracy: 0.5645 - val_loss: 1.2690
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 538ms/step - accuracy: 0.5271 - loss: 1.3012 - val_accu

Model training finished.
Model saved to /home/qod110/Documents/Project/2nd_ai_web_project/ai_model/cat_emotion_mobilenet_v2_v1_test.h5
